# Lecture 2 - Kinematics

In [ ]:
import numpy as np

filename = "data/Three-particles-1.dat"
data = np.loadtxt(filename)
data.shape

In [ ]:
n_final_state = 3
pa, p1, p2, p3 = (data[i::4].T for i in range(n_final_state + 1))

In [ ]:
def mass(p: np.ndarray) -> np.ndarray:
    return np.sqrt(mass_squared(p))


def mass_squared(p: np.ndarray) -> np.ndarray:
    return p[0] ** 2 - np.sum(p[1:] ** 2, axis=0)


p0 = p1 + p2 + p3
m0 = mass(p0)
print(f"{m0.mean():.4g} +/- {m0.std():.4g}")

In [ ]:
from IPython.display import Math

for i, p in enumerate([p0, p1, p2, p3]):
    display(Math(Rf"m_{i} = {mass(p).mean():.3g}\text{{ GeV}}"))

display(Math(Rf"m_{{a}} = {mass(pa).mean():.3g}\text{{ GeV}}"))

In [ ]:
from IPython.display import Markdown
from particle import Particle


def find_candidates(m: float, delta: float = 0.001) -> list[Particle]:
    return Particle.findall(lambda p: (m - delta) < 1e-3 * p.mass < (m + delta))


m1 = mass(p1).mean()
m2 = mass(p2).mean()
m3 = mass(p3).mean()
particles = tuple(find_candidates(m.mean())[0] for m in [m1, m2, m3])

src = "Final state: $" + ", ".join(f"{p.latex_name}" for p in particles) + "$"
Markdown(src)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.hist2d(
    mass_squared(p1 + p2),
    mass_squared(p2 + p3),
    bins=100,
    cmin=1,
)
ax.set_xlabel(R"$s_{12}\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$s_{23}\;\left[\mathrm{GeV}^2\right]$")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(
    mass_squared(p1 + p2),
    mass_squared(p2 + p3),
    c="black",
    s=1e-3,
)
ax.set_xlabel(R"$s_{12}\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$s_{23}\;\left[\mathrm{GeV}^2\right]$")
plt.show()